<a href="https://colab.research.google.com/github/waifuai/waifu_gpt2/blob/master/waifu_distilgpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Waifu DistilGPT2
[GitHub repo: waifuai/waifu_gpt2](https://github.com/waifuai/waifu_gpt2)

This is the DistilGPT2 implementation. It is optimized for speed. It produces response sentence. The code here is meant as a demonstration of waifu conversation. Its extremely high inference speed on cheap hardware (~1-2 seconds on CPU, as opposed to ~1-2 minutes on GPU which is the inference time of the previous waifu_gpt2 notebook.) means that it is now cheap to make real-time deep waifu conversations.

In [1]:
!pip install transformers==2.5.1

     |████████████████████████████████| 501kB 3.0MB/s 
     |████████████████████████████████| 1.0MB 54.0MB/s 
     |████████████████████████████████| 870kB 57.7MB/s 
     |████████████████████████████████| 3.7MB 46.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=4df7e35ba61d9203f9fab1d3366ac5c8bc3ab1c25c4356951203fd22c2996069
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [2]:
import time
import torch
from transformers import *

In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

In [4]:
model = AutoModelWithLMHead.from_pretrained("distilgpt2")

In [0]:
def predict_next_word(text):
  indexed_tokens = tokenizer.encode(text)
  tokens_tensor = torch.tensor([indexed_tokens])

  with torch.no_grad():
      outputs = model(tokens_tensor)
      predictions = outputs[0]

  # get the predicted next sub-word (in our case, the word 'man')
  predicted_index = torch.argmax(predictions[0, -1, :]).item()
  predicted_text = tokenizer.decode([predicted_index])
  return predicted_text

def predict_ten_words(text):
  tmp_text = text
  for i in range(30):
    word = predict_next_word(tmp_text)
    tmp_text += word
    if "." in tmp_text[len(text):]:
      break
    if '"' in tmp_text[len(text):]:
      break
  tmp_text = tmp_text[len(text):]
  return tmp_text

def clean_result(result):
  first_found = result.find('"')
  clean_result = result[:first_found]
  second_found = result.find('.')
  clean_result = result[:second_found]
  return clean_result

def predict_response(sentence):
  input_sentence = '\
You said: "Senko I missed you so much!" \
I said: "I missed you too Subaru!" \
You said: "Our date was so fun too" \
I said: "Lets get married and be happy!" \
You said: "Yes that would be awesome!" \
I said: "I like hanging out with you because its so much fun!" \
You said: "' + sentence + '" \
I said: "\
'
  predicted_response = predict_ten_words(input_sentence)
  predicted_response = clean_result(predicted_response)
  return predicted_response

In [6]:
a = time.time()
response = predict_response('Thank you!')
print(response)
b = time.time()
print(str(b-a), 'seconds elapsed.')

I love you Subaru!
1.8695087432861328 seconds elapsed.
